# Grundrechenarten im Erweiterungskörper $\mathbb{F}_q$

### Imports der benötigten Module

In [1]:
%run Module_DefinierendeRelationen.ipynb

importing notebook from DefinierendeRelationen.ipynb
importing notebook from Polynomdivision.ipynb
importing notebook from FpRechner.ipynb


In [118]:
import numpy as np
from IPython.display import display, Math

In [4]:
import Polynomdivision as polydiv
import FpRechner as Fp
import DefinierendeRelationen as defrel

### Hilfsfunktionen

$polyproduct\_number(pol1,a,p)$ nimmt ein Polynom $pol$ (oder Element eines Erweiterungskörpers $mathbb F_q$, denn universell einsetzbar) in Tupelschreibweise und multipliziert es mit einer Zahl $a \in \mathbb F_q$ 

In [111]:
def polyproduct_number(pol,a,p):
    return tuple((np.array(pol)*a)%p)

$polyproduct\_poly(pol1,pol2,p)$ nimmt ein Polynom $pol1$ und multipliziert es mit einem anderen Polynom $pol2$, beide in Tupelschreibweise.  

In [34]:
def polyproduct_poly(pol1,pol2,p):
    l1 = len(pol1)
    l2 = len(pol2)
    lges = l1+l2-1
    result = np.zeros(lges, dtype=int)
    pol1=np.array(pol1)
    pol2=np.array(pol2)
    for i in range(l2):
        temp = polydiv.shiftleft(pol1,l2-i-1)
        temp = polyproduct_number(temp,pol2[i],p) 
        temp = polydiv.fill(temp,lges-len(temp))
        result = (result+temp)%p
    return tuple(result)
    

In [41]:
polyproduct_poly((1,1,0),(2,2,1),3)

(2, 1, 0, 1, 0)

### Addition in $\mathbb{F}_q$ 

$summe(a,b,p)$ erhält zwei Elemente $a$ und $b$ eines Erweiterungskörpers $\mathbb F_q$ mit $q=p^l$ und gibt deren Summe zurück. Hierbei geben wir der Funktion Element $a$ definiert als $a_{l-1}\cdot \alpha^{l-1}+a_{l-2}\cdot \alpha^{l-2}+\cdots+a_2\cdot \alpha^2+a_1\cdot \alpha+ a0$ mit $a_0,...,a_{l-1} \in \mathbb F_p$ in der Tupelschreibweise $a=$($a_{l-1}$,$a_{l-2}$,$\cdots$,$a_2$,$a_1$,$a_0$) mit und Element $b$ ebenso. Hierbei muss darauf geachtet werden dass auch alle Nullen in das Tupel geschrieben werden, und beide Tupel die gleiche länge haben müssen, da in den Funktionen über die Länge der Tupel der Grad des Körpers ermittelt wird. 

In [15]:
def summe(a,b,p):
    return tuple((np.array(a)+np.array(b))%p)

### Subtraktion in $\mathbb{F}_q$ 

$differenz(a,b,p)$ erhält die beiden Elemente $a$ und $b$ ebenfalls in der Tupelschreibweise und rechnet dann $a-b$, wobei $a,b \in \mathbb F_q$ mit $q=p^l$

In [17]:
def differenz(a,b,p):
    return tuple((np.array(a)-np.array(b))%p)

### Multiplikation in $\mathbb{F}_q$ 

$produkt(a,b,def\_rel,p)$ berechnet das Produkt zweier Elemente $a,b \in \mathbb F_q$, welche in Tupelschreibweise mitgegeben werden.
    Zusätzlich benötigt die Funktion noch eine definierende Relation in Tupelschreibweise $def\_rel=(r_{l-1},r_{l-2},...,r_2,r_1,r_0)$, welche den Körper $\mathbb F_q$ mit $q=p^l$ beschreibt.

In [108]:
def produkt(a,b,def_rel,p):
    prod = polyproduct_poly(a,b,p)
    l = len(def_rel)
    if len(prod) == l:
        return prod
    ergebnis = np.array(prod[-l:])
    überhang = np.array(prod[:-l])
    while len(prod)>l:
        prod = polyproduct_poly(überhang,def_rel,p)
        ergebnis = (ergebnis + np.array(prod[-l:]))%p
        überhang = (np.array(prod[:-l]))%p
    return tuple(ergebnis)

### Division in $\mathbb{F}_q$

#### Inverses Element

$inverses(x,def_rel,p)$ erhält ein Element $x \in \mathbb F_q$ mit $q=p^l$ in Tupelschreibweise und eine definierende Relation $def_rel$ welche $\mathbb F_q$ definiert in Tupelschreibweise und gibt das Inverse Element zurück. 

In [61]:
def inverses(x,def_rel,p):
    if x==(1,):
        return (1,)
    
    a0 = np.array((1,)) 
    b0 = np.array((0,))
    a1 = np.array((0,))
    b1 = np.array((1,))    
    r0 = np.array(defrel.relation_to_polynom(def_rel,p))
    r1 = np.array(x)
    
    while(np.sum(r1)!=0):
        q = polydiv.poldiv(r0,r1,p)[0]
        rtemp = r0
        r0 = r1
        r1 = polydiv.poldiv(rtemp, r1,p)[1]
        atemp = a0
        a0 = a1
        l = len(atemp)-len(polyproduct_poly(q,a1,p))
        if l<0:
            a1 = (polydiv.fill(atemp,(-1)*l) - polyproduct_poly(q,a1,p))%p
        elif l==0: 
            a1 = (atemp - polyproduct_poly(q,a1,p))%p
        else:
            a1 = (atemp - polydiv.fill(polyproduct_poly(q,a1,p)))%p
        btemp = b0
        b0 = b1
        l = len(btemp)-len(polyproduct_poly(q,b1,p))
        if l<0:
            b1 = (polydiv.fill(btemp,(-1)*l) - polyproduct_poly(q,b1,p))%p
        elif l==0: 
            b1 = (btemp - polyproduct_poly(q,a1,p))%p
        else:
            b1 = (btemp - polydiv.fill(polyproduct_poly(q,b1,p)))%p
    
    return polyproduct_number(b0,Fp.inverses(r0[-1],p),p)

#### Division

$divide(a,b,def\_rel,p)$ berechnet $\frac{a}{b}$ innerhalb des Körpers $\mathbb F_q$ mit $q=p^l$ welcher über $def\_rel$ definiert wird.

In [112]:
def divide(a,b,def_rel,p):
    inv = inverses(b,def_rel,p)
    return produkt(a,inv,def_rel,p)

### Print Funktion

$print\_element(element)$ erhält ein Element aus $\mathbb F_q$ in Tupelschreibweise und gibt es in der mathematischen Schreibweise aus. 

In [122]:
def print_element(element):
    l = len(element)
    s = r''
    for i in range(l-2):
        if element[i]>1:
            s = s+(r''+str(element[i])+r' \alpha^{'+str(l-i-1)+'}+')
        elif element[i]==1:
            s = s+(r' \alpha^{'+str(l-i-1)+r'}+')
    if element[l-2]>1:
        s = s+(r''+str(element[l-2])+r' \alpha+')
    elif element[l-2]==1:
        s = s+(r' \alpha+')
    if element[l-1]>0:
        s=s+(r''+str(element[l-1])+r'')
    else:
        s=s[:-1]
    display(Math(s))